In [1]:
import pandas as pd
from pathlib import Path

def import_data(filename):
    # ".." = remonter d'un niveau
    data_path = Path("..") / "Data" / filename
    return pd.read_csv(data_path, header=1)

def import_geojson(filename):
    data_path = Path("..") /Path("Data") / filename
    return gpd.read_file(data_path)

def rename_and_transpose(df):
    # 1. Renommer la première colonne
    df.columns.values[0] = "Year"
    
    # 2. Transposer
    df_t = df.transpose()
    
    # 3. Mettre la première ligne comme en-tête
    df_t.columns = df_t.iloc[0]     # La première ligne devient l’en-tête
    df_t = df_t.drop(df_t.index[0]) # Supprimer cette ligne des données
    
    return df_t

def rename_and_transpose2(df):
    # 1. Renommer la première colonne
    df.columns.values[0] = "Year"
    
    # 2. Transposer
    df_t = df.transpose()
    
    # 3. Mettre la première ligne comme en-tête
    df_t.columns = df_t.iloc[0]     # La première ligne devient l’en-tête
    df_t = df_t.drop(df_t.index[0]) # Supprimer cette ligne des données
    
    return df_

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import TwoSlopeNorm, LinearSegmentedColormap
import matplotlib.cm as cm
import pandas as pd

def plot_world_map(df, gdf, year, country_col="NAME"):
    """
    Carte du monde pour une année donnée.
    Bleu → négatif, Blanc → 0, Rouge → positif
    Pays sans données en gris clair avec patch centré sous la colorbar.
    """
    year = str(year)

    # Préparer les données
    df_year = df[[year]].copy()
    df_year = df_year.reset_index().rename(columns={"index": country_col, year: "value"})
    df_year["value"] = pd.to_numeric(df_year["value"], errors='coerce')

    # Fusion avec le GeoDataFrame
    gdf_merged = gdf.merge(df_year, on=country_col, how="left")

    # Création de la figure
    fig, ax = plt.subplots(figsize=(14, 7))

    # Colormap bleu → blanc → rouge
    colors = [(0, 0, 0.8), (1, 1, 1), (0.8, 0, 0)]
    cmap = LinearSegmentedColormap.from_list("blue_white_red", colors, N=256)

    # Normalisation avec blanc à 0
    vmin = gdf_merged["value"].min()
    vmax = gdf_merged["value"].max()
    norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

    # Tracer la carte
    gdf_merged.plot(column="value",
                    cmap=cmap,
                    norm=norm,
                    linewidth=0.5,
                    edgecolor="black",
                    missing_kwds={"color": "lightgrey"},
                    ax=ax)

    # Colorbar simple
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, orientation="vertical",
                        fraction=0.025, pad=0.02)
    cbar.set_label("TgC/year", fontsize=12)

    # Patch gris "Sans données"
    patch = mpatches.Patch(color="lightgrey", label="Sans données")
    cbar.ax.legend(handles=[patch],
                   loc='upper center',
                   bbox_to_anchor=(0.5, -0.1),
                   frameon=False,
                   fontsize=10)

    # Titre
    ax.set_title(f"Net Emission in {year}", fontsize=16, pad=15)
    ax.axis("off")

    plt.show(block=True)


In [5]:
df = import_data("GCB_NetEmissions_Countries.csv")
df_t = rename_and_transpose(df)

# Garder seulement les colonnes dont le nom est convertible en entier (les années)
df_t = df_t[[c for c in df_t.columns if str(c).replace('.0','').isdigit()]]

# Convertir les colonnes en int
df_t.columns = [int(str(c).replace('.0','')) for c in df_t.columns]


df_t.head()


,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Afghanistan,4.17891,3.59662,3.32416,3.23023,2.79609,2.84472,2.62185,2.30549,2.17469,2.01775,...,0.2982,0.28716,0.35204,1.02206,0.69029,1.35329,0.983,1.33826,1.5648,1.69074
Albania,0.98295,0.7255,0.60905,0.46588,0.65877,0.70381,0.716,0.60361,0.58366,0.57642,...,0.05507,0.17305,0.03659,0.04546,-0.01066,-0.04848,-0.09121,-0.1042,-0.13681,-0.17435
Algeria,4.38642,1.8356,0.8065,0.2254,0.33382,0.09481,0.57291,-0.02041,-0.1056,0.0305,...,-0.00398,0.0206,0.06336,0.07388,0.29755,0.14063,0.16296,0.20926,0.2678,0.2807
Andorra,0.00563,0.0074,0.00565,0.00443,0.00364,0.00305,0.00272,0.00202,0.00252,0.00178,...,-0.00144,-0.00155,-0.00155,-0.00158,-0.00188,-0.00201,-0.00203,-0.00197,-0.00198,-0.002
Angola,25.0768,17.26244,12.35364,10.40859,8.94263,8.1837,7.5842,7.22189,7.20413,6.35542,...,29.27385,30.00732,27.66582,26.76205,25.13832,24.01049,23.17754,23.47849,23.32447,23.35756


In [11]:
gdf = import_geojson("world-administrative-boundaries.geojson")
gdf.head()


,geo_point_2d,iso3,status,color_code,name,continent,region,iso_3166_1_alpha_2_codes,french_short,geometry
0,"{ ""lon"": -16.984917430414384, ""lat"": 32.747916...",None,PT Territory,PRT,Madeira Islands,Europe,Southern Europe,None,Madeira Islands,"POLYGON ((-17.1025 32.82333, -17.05306 32.8094..."
1,"{ ""lon"": 33.743791080217562, ""lat"": 21.8927401...",None,Adm. by EGY,EGY,Ma'tan al-Sarra,Africa,Northern Africa,None,Ma'tan al-Sarra,"POLYGON ((33.25104 21.99977, 34.15064 21.99603..."
2,"{ ""lon"": 9.5613358449883421, ""lat"": 34.1108585...",TUN,Member State,TUN,Tunisia,Africa,Northern Africa,TN,Tunisie,"MULTIPOLYGON (((10.99361 33.75, 10.93778 33.72..."
3,"{ ""lon"": 43.77213543247138, ""lat"": 33.04802449...",IRQ,Member State,IRQ,Iraq,Asia,Western Asia,IQ,Iraq,"POLYGON ((44.78734 37.14971, 44.76617 37.11228..."
4,"{ ""lon"": -6.3178452255610269, ""lat"": 31.883624...",MAR,Member State,MAR,Morocco,Africa,Northern Africa,MA,Maroc,"POLYGON ((-2.94694 35.32916, -2.96618 35.31663..."


In [81]:

# Tracé de la carte pour l’année choisie
plot_world_map(df_t, gdf, year=1990, country_col="name")
  # adapte le nom du champ pays si nécessaire


In [184]:
import folium
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import branca.colormap as bcm
import tempfile
import webbrowser

def plot_world_map_interactive(df, gdf, year=2020, country_col="NAME", title=None, unit="TgC/year"):
    """
    Create an interactive Folium map for a specific year showing net emissions per country.

    Parameters:
    - df : pd.DataFrame
        DataFrame with country emissions data. Must have a column with the year as int.
    - gdf : gpd.GeoDataFrame
        GeoDataFrame with geometries for countries.
    - year : int, default 2020
        Year of data to display.
    - country_col : str, default "NAME"
        Column in gdf containing country names.
    - title : str, optional
        Title of the map.
    - unit : str, default "TgC/year"
        Unit of the values displayed.
    """
    
    if title is None:
        title = f"Land Use Change - Net emission per country in {year}"

    # Prepare GeoDataFrame by merging data
    gdf_merged = gdf.copy()
    gdf_merged = gdf_merged.set_index(country_col)
    
    df_numeric = df[[year]].apply(pd.to_numeric, errors='coerce')
    gdf_merged[year] = df_numeric[year]

    # Colormap
    vmin = df_numeric.min().min()
    vmax = df_numeric.max().max()
    cmap = plt.get_cmap("bwr")
    norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

    # Create Folium map
    m = folium.Map(location=[20, 0], zoom_start=2, tiles="cartodbpositron")

    # Add countries
    for idx, row in gdf_merged.iterrows():
        val = row[year]
        color = "#D3D3D3" if pd.isna(val) else colors.to_hex(cmap(norm(val)))
        popup_text = f"<b>{idx}</b><br>{val:.2f} {unit}" if pd.notna(val) else f"<b>{idx}</b><br>No data"

        folium.GeoJson(
            row["geometry"],
            style_function=lambda feature, fill=color: {
                "fillColor": fill,
                "color": "black",
                "weight": 0.5,
                "fillOpacity": 0.7
            },
            tooltip=folium.Tooltip(popup_text, sticky=True)
        ).add_to(m)

    # Add color legend
    colormap = bcm.LinearColormap(colors=["blue", "white", "red"], vmin=vmin, vmax=vmax, caption=unit)
    colormap.add_to(m)

    # Add title
    m.get_root().html.add_child(
        folium.Element(f'<h3 align="center" style="font-size:20px"><b>{title} [{unit}]</b></h3>')
    )

    # Save and open in browser
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".html")
    m.save(tmp.name)
    webbrowser.open(f"file://{tmp.name}")

    return m


In [186]:
# Example usage:

# df: DataFrame with columns like ['Country', 2020, 2021, ...]
# gdf: GeoDataFrame with geometries and a column 'NAME' for country names

map_2020 = plot_world_map_interactive(
    df=df_t,            # your data DataFrame
    gdf=gdf,          # your GeoDataFrame
    year=2020,        # year to display
    country_col="name",  # column in gdf with country names
    title="Net Carbon Emissions 2020",  # optional title
    unit="TgC/year"       # optional unit
)


In [7]:
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Données
df = pd.DataFrame({
    "Année": [2020, 2021, 2022, 2023],
    "Valeur": [10, 15, 13, 18]
})

# Crée deux figures séparées avec Plotly Express
fig1 = px.line(df, x="Année", y="Valeur", title="Évolution")
fig2 = px.bar(df, x="Année", y="Valeur", title="Histogramme")

# Combine-les dans un subplot (2 lignes, 1 colonne)
fig = make_subplots(rows=2, cols=1, subplot_titles=("Évolution", "Histogramme"))

# Ajoute les traces de chaque figure
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

for trace in fig2.data:
    fig.add_trace(trace, row=2, col=1)

# Mets à jour la mise en page
fig.update_layout(height=800, title_text="Deux graphiques combinés")

# Enregistre dans un HTML et ouvre dans le navigateur
fig.write_html("graphiques_combines.html", auto_open=True)
